In [7]:
import os
kaggle_data={"username":"ahmedmagdy123","key":"b5b25f0f47116bfce5fe5616190fcb36"}
os.environ['KAGGLE_USERNAME']=kaggle_data["username"]
os.environ['KAGGLE_KEY']=kaggle_data["key"]

In [8]:
!kaggle competitions download -c mct-aic-2

 98%|███████████████████████████████████████▎| 236M/240M [00:07<00:00, 32.5MB/s]
100%|████████████████████████████████████████| 240M/240M [00:07<00:00, 35.0MB/s]


In [9]:
import zipfile

zip_file_path = '/kaggle/working/mct-aic-2.zip'
extract_to_path = '/kaggle/working/mct-test_data/'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of zip file into the specified directory
    zip_ref.extractall(extract_to_path)

print(f'Files have been extracted to {extract_to_path}')

Files have been extracted to /kaggle/working/mct-test_data/


In [3]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget -y
!apt-get install sox libsndfile1 ffmpeg -y
!pip install text-unidecode -y
!pip install matplotlib>=3.3.2 -y



"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
exit()


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-7ubuntu0.2).
libsndfile1 set to manually installed.
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 64 not 

In [3]:
## Install NeMo
BRANCH = 'main'
!pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH


  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-req-build-bjg34bj8
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-req-build-bjg34bj8
  Resolved https://github.com/NVIDIA/NeMo.git to commit 043a0801a64ed40245a6d069d0eaa1c6ed7465cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install nemo_toolkit['asr']

In [5]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

In [6]:
# Import necessary module
from nemo.collections.asr.models import EncDecCTCModel

# Path to your saved checkpoint file
checkpoint_path = '/kaggle/input/thiryepochmodel/thirty.nemo' # Changed to a file path

# Restore the model from the checkpoint
thirty_epochs_model = EncDecCTCModel.restore_from(restore_path=checkpoint_path)

[NeMo W 2024-07-02 16:19:56 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /kaggle/input/manifests/train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - ء
    - ا
    - ب
    - ت
    - ث
    - ج
    - ح
    - خ
    - د
    - ذ
    - ر
    - ز
    - س
    - ش
    - ص
    - ض
    - ط
    - ظ
    - ع
    - غ
    - ف
    - ق
    - ك
    - ل
    - م
    - ن
    - ه
    - و
    - ي
    - ''''
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 4
    pin_memory: true
    /kaggle/input/manifests/train_manifest.json: /kaggle/input/manifests/train_manifest.json
    
[NeMo W 2024-07-02 16:19:56 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid co

[NeMo I 2024-07-02 16:19:56 features:305] PADDING: 16
[NeMo I 2024-07-02 16:19:56 save_restore_connector:272] Model EncDecCTCModel was successfully restored from /kaggle/input/thiryepochmodel/thirty.nemo.


In [10]:

import os
import pandas as pd

# Assuming you have a list of WAV files in your test directory
test_dir = '/kaggle/working/mct-test_data/test'
wav_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith('.wav')]

transcriptions = []
for wav_file in wav_files:
    audio_id = os.path.splitext(os.path.basename(wav_file))[0]  # Get audio file ID without extension
    transcription = thirty_epochs_model.transcribe(audio=wav_file)  # Replace with your actual transcription function
    transcriptions.append({'audio': audio_id, 'transcript': transcription})

# Create a DataFrame from transcriptions
df = pd.DataFrame(transcriptions)

# Save DataFrame to CSV
submission_file = '/kaggle/working/submission.csv'  # Specify your desired output file path
df.to_csv(submission_file, index=False)

Transcribing: 100%|██████████| 1/1 [00:00<00:00, 12.01it/s]
